# Session 3: 상관관계와 예측 모델 (Integrated)

## 🎯 학습 목표
- **Deep Theory**: 공분산과 상관계수, 그리고 최소제곱법(OLS)의 미분 과정을 이해합니다.
- **Practice**: 경사하강법(Gradient Descent)을 직접 구현하여 선형 회귀 모델을 학습시킵니다.
- **Concepts**: 상관관계와 인과관계를 구분하고, 선형 모델의 한계를 배웁니다.
- **Experiment**: 노이즈가 모델 학습에 미치는 영향을 실험합니다.

---

# Part 1. 📐 Deep Theory: 관계와 학습

## 1. 공분산(Covariance)과 상관계수(Correlation)

두 변수 $X, Y$가 함께 변하는 정도를 측정합니다.

$$ Cov(X, Y) = \frac{1}{n-1} \sum (x_i - \bar{x})(y_i - \bar{y}) $$

- **공분산의 문제**: $X, Y$의 단위(Scale)에 따라 값이 너무 커지거나 작아집니다.
- **해결 (정규화)**: 각 변수의 표준편차로 나누어 $-1 \sim 1$ 사이 값으로 맞춥니다. -> **상관계수($r$)**

$$ r_{xy} = \frac{Cov(X, Y)}{s_x s_y} $$

## 2. 최소제곱법 (Ordinary Least Squares)

데이터 $y$와 예측 $\hat{y} = wx + b$ 사이의 오차 제곱합(SSE)을 최소화하는 $w, b$를 찾습니다.

$$ Loss = \sum (y_i - (wx_i + b))^2 $$

이 Loss 함수는 아래로 볼록한(Convex) 2차 함수이므로, **미분해서 0이 되는 지점**이 최솟값입니다.

$$ \frac{\partial Loss}{\partial w} = -2 \sum x_i(y_i - (wx_i + b)) = 0 $$
$$ \frac{\partial Loss}{\partial b} = -2 \sum (y_i - (wx_i + b)) = 0 $$

---
# Part 2. 💻 Manual Implementation: 경사하강법 (Gradient Descent)

미분 공식을 이용하여, 조금씩 경사를 타고 내려가 최적의 $w, b$를 찾는 알고리즘을 구현합니다.
$$ w_{new} = w_{old} - \alpha \frac{\partial Loss}{\partial w} $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 가상의 데이터 생성 (y = 2x + 1 + noise)
np.random.seed(42)
X = np.random.rand(100, 1)
y = 2 * X + 1 + np.random.randn(100, 1) * 0.1

def gradient_descent(X, y, lr=0.1, epochs=1000):
    w = np.random.randn(1, 1)
    b = np.random.randn(1, 1)
    N = len(y)
    
    for _ in range(epochs):
        # 예측
        y_pred = X * w + b
        
        # 편미분 (Gradient 계산)
        # dLoss/dw = -2/N * sum(x * error)
        error = y - y_pred
        dw = -2 * np.sum(X * error) / N  # 평균제곱오차 기준
        db = -2 * np.sum(error) / N
        
        # 파라미터 업데이트
        w = w - lr * dw
        b = b - lr * db
        
    return w, b

w_hat, b_hat = gradient_descent(X, y)
print(f"Estimated: w={w_hat[0][0]:.4f}, b={b_hat[0][0]:.4f} (Actual: w=2, b=1)")

---
# Part 3. 📖 Concepts & Practice: 상관 분석

### 히트맵(Heatmap)으로 상관관계 시각화
변수가 많을 때는 히트맵이 유용합니다.

In [ ]:
import seaborn as sns
import pandas as pd

# 예제 데이터프레임
df = pd.DataFrame({'X': X.flatten(), 'Y': y.flatten()})
df['Z'] = np.random.rand(100) # 아무 관련 없는 변수

corr = df.corr()

plt.figure(figsize=(6, 5))
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlation Heatmap")
plt.show()

---
# Part 4. 🧪 Experiment: 노이즈와 모델

### Scenario: 노이즈가 많아지면 학습이 어떻게 될까?
데이터에 잡음(Noise)이 커질수록 회귀선이 정답(Truth)에서 멀어지고 신뢰도가 떨어지는 것을 확인합니다.

In [ ]:
noise_levels = [0.1, 0.5, 1.0, 2.0]

plt.figure(figsize=(15, 3))

for i, noise in enumerate(noise_levels):
    # 노이즈 추가
    y_noisy = 2 * X + 1 + np.random.randn(100, 1) * noise
    
    # 학습
    w_n, b_n = gradient_descent(X, y_noisy)
    
    # 시각화
    plt.subplot(1, 4, i+1)
    plt.scatter(X, y_noisy, alpha=0.5, s=10)
    plt.plot(X, X*w_n + b_n, color='red')
    plt.title(f"Noise={noise}\nw={w_n[0][0]:.2f}")

plt.tight_layout()
plt.show()

> **Discussion**: 노이즈가 커질수록 데이터가 퍼지고, $w$값의 변동성도 커집니다. (모델의 Variance 증가)